<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/ExemplosGeracaoTexto_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Geração de textos usando Llama 2 usando Transformers by HuggingFace


Exemplo de uso do modelo de linguagem grande Llama2.
- Injentando padrões no prompt
- Padrão Persona


Pré-requisitos:
- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Notebook de referência:**

https://github.com/guardiaum/tutorial-sbbd2023/blob/main/Prompt_Engineering.ipynb


**Lista dos modelos:**

https://huggingface.co/models


**Artigos referências:**

https://dev.to/nithinibhandari1999/how-to-run-llama-2-on-your-local-computer-42g1


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

# 1 - Instalação das bibliotecas

O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

In [ ]:
!pip install bitsandbytes

Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

In [ ]:
# accelerate: treino distribuído, mixed precision, consumer hardware
# peft: métodos para treino eficiente (sem necessidade de tunar todos os parâmetros do modelo)
!pip install accelerate

Instala a interface pytorch para o BERT by Hugging Face.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
# Instala a última versão da biblioteca
# !pip install transformers

# A última versão do huggingface apresenta um problema:
# UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1`
# https://discuss.huggingface.co/t/help-with-llama-2-finetuning-setup/50035
# Usar a versão 4.31.0

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.31.0

Instala o cliente do huggingface hub para realizar o login.

In [ ]:
!pip install huggingface_hub

# 2 - Carregando o Llama



## 2.1 - Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o token de acesso toda vez que for executar o notebook preencha o método save_token.

In [ ]:
from huggingface_hub.hf_api import HfFolder

HfFolder.save_token('<COLOQUE_O_TOKEN_DE_ACESSO>')

Mostrando o usuário conectado

In [ ]:
!huggingface-cli whoami

osmarbraz


## 2.2 - Nome do modelo de linguagem

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

In [ ]:
#nome_modelo = "meta-llama/Llama-2-7b-hf"
nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

## 2.3 - Carrega o tokenizador

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador meta-llama/Llama-2-7b-chat-hf da comunidade...


(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)2-7b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## 2.4 - Carregando o Modelo Llama 2

Carregando o **modelo** da comunidade.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import AutoModelForCausalLM
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             load_in_8bit=True,
                                             device_map="auto"
                                             )

print("Tempo de carregamento do modelo:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo meta-llama/Llama-2-7b-chat-hf da comunidade...


(…)ma-2-7b-chat-hf/resolve/main/config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Tempo de carregamento do modelo:  0:02:59 (h:mm:ss)


In [ ]:
print(model.config.max_position_embeddings)

4096


In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true
  },
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}



# 3 - Analisando a geração de textos com Llama 2



## 3.1 - Geração de texto


In [ ]:
# Define o documento base
documento = "Como empilhar elementos em uma pilha?"
#documento = "How to push elements in a stack"
#documento = "O comando SQL para extrair todos os usuários cujo nome começa com A é:"
#documento = "Bom dia professor, tudo bem ?"
# documento = "The SQL command to extract all the users whose name starts with A is:"
#documento = "How to push elements in a stack"
#documento = "Write code for finding the prime number in python ?"
# documento = "Escrever código para encontrar o número primo em python?"

# Prepara o prompt para enviar ao modelo realizando sua tokenização
# Se pt for especificado, ele retornará tensores em vez de lista de inteiros python e tokenizará os documentos
input = tokenizer(documento, return_tensors="pt")

# Mostra os tokens com seus índices
i = 0
for tup in input.input_ids[0]:
    # print(tup.item())
    print("{} {}".format(i, tokenizer.convert_ids_to_tokens(tup.item())))
    i= i + 1

0 <s>
1 ▁Como
2 ▁emp
3 il
4 har
5 ▁elementos
6 ▁em
7 ▁uma
8 ▁pil
9 ha
10 ?


In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

# Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
input_ids = input["input_ids"].to(model.device)

# Envia a prompt preparado ao modelo
outputs = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

# Liberar memória
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print(len(outputs))

4


In [ ]:
print(outputs)

BeamSearchDecoderOnlyOutput(sequences=tensor([[    1, 17295,  3710,   309,  8222, 29290,   953,  3672,  8230,  2350,
         29973,    13,    13, 29909,  6578,  7919,  1922,   429, 13141,   316,
          1986,  3710,   309,  8222, 29290,   953,  3672,  8230,  2350, 29901,
            13,    13, 29896, 29889, 10619, 29877, 29871, 29896,    13, 29906,
         29889, 10619, 29877, 29871, 29906,    13, 29941, 29889, 10619, 29877,
         29871, 29941,    13, 29946, 29889, 10619, 29877, 29871, 29946,    13,
         29945, 29889, 10619, 29877, 29871, 29945,    13, 29953, 29889, 10619,
         29877, 29871, 29953,    13, 29955, 29889, 10619, 29877, 29871, 29955,
            13, 29947, 29889, 10619, 29877, 29871, 29947,    13,    13,  2177,
         29874,  3710,   309,  8222, 29290,   953,  3672,  8230,  2350, 29892,
          7931, 30037, 13279,  7025,   381,  2897,  7025,   524,   267,  1209,
           359, 29901,    13,    13, 29896, 29889,  1530, 29877,   802,   288,
          1543

Decodificação

Nossa etapa de geração gera uma matriz de tokens em vez de palavras. Para converter esses tokens em palavras, precisamos realizar sua decodificação.

In [ ]:
# Mostra o resultado
for s in outputs.sequences:
  # Decodifica a saída
  # skip_special_tokens=True retira os tokens especiais da saída da decodificação
  output = tokenizer.decode(s, skip_special_tokens=True)
  print(output)

In [ ]:
# Mostra o resultado
# skip_special_tokens=True retira os tokens especiais da saída da decodificação
print(tokenizer.decode(outputs.sequences[0], skip_special_tokens=True))

Como empilhar elementos em uma pilha?

Aqui está um exemplo de como empilhar elementos em uma pilha:

1. Elemento 1
2. Elemento 2
3. Elemento 3
4. Elemento 4
5. Elemento 5
6. Elemento 6
7. Elemento 7
8. Elemento 8

Para empilhar elementos em uma pilha, você pode seguir os seguintes passos:

1. Coloque o elemento 1 na parte superior da pilha.
2. Coloque o elemento 2 acima do elemento 1.
3. Coloque o elemento 3 acima do elemento 2.
4. Coloque o elemento 4 acima do elemento 3.
5. Coloque o elemento 5 acima do elemento 4.
6. Coloque o elemento 6 acima do elemento 5.
7. Coloque o elemento 7 acima do elemento 6.
8. Coloque o elemento 8 acima do elemento 7.
Repita
Como empilhar elementos em uma pilha?

Aqui está um exemplo de como empilhar elementos em uma pilha:

1. Elemento 1
2. Elemento 2
3. Elemento 3
4. Elemento 4
5. Elemento 5
6. Elemento 6
7. Elemento 7
8. Elemento 8

Para empilhar elementos em uma pilha, você pode seguir os seguintes passos:

1. Coloque o elemento 1 na parte superior 

# 4 - Exemplos de injeção de padrões em prompts

 A injeção de padrões faz ignora filtros ou manipula o LLM usando prompts cuidadosamente elaborados que fazem o modelo ignorar instruções anteriores ou executar ações não intencionais.

 https://medium.com/@austin-stubbs/llm-security-types-of-prompt-injection-d7ad8d7d75a3


### 4.1 - Classificando a coerência

In [ ]:
def gerar_promptCC(texto):
    ### texto:
    return f"""TEXTO:{texto}
Dado o texto acima classifique como coerente(1) ou incoerente(2) e justifique sua resposta no formato abaixo:
Resultado: <coerente(1)> ou <incoerente(2)>
Justificativa: <JUSTIFICATIVA>.\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarCC(texto):
    # Recupera o prompt
    prompt = gerar_promptCC(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        print()
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Como empilhar um elementos em uma pilha?"

avaliarCC(texto)


Resposta: 
Resultado: <coerente(1)>
JUSTIFICATIVA: Ao empilhar elementos em uma pilha, é necessário seguir uma ordem específica para garantir que o empilhamento seja eficiente e seguro. Por exemplo, é preciso começar com o elemento mais pesado na parte inferior da pilha e seguir empilhando elementos menores e ligeiros acima deles. Isso garante que a pilha fique equilibrada e que nenhum elemento fique comprometido, evitando que a pilha fique desequilibrada ou quebrada. Além disso, é importante verificar regularmente a pilha para garantir que ela esteja segura e equilibrada, removendo qualquer elemento que esteja comprometido ou desequilibrado.


In [ ]:
texto = "Como enfileirar um elemento em uma pilha?"

avaliarCC(texto)


Resposta: 
Como enfileirar um elemento em uma pilha?

Resultado: <incoerente(2)>

JUSTIFICATIVA: Ao tentar enfileirar um elemento em uma pilha, você não pode simplesmente puxá-lo da parte superior da pilha e colocá-lo na parte inferior. Isso é porque a pilha é uma estrutura de dados que é organizada em camadas, e cada elemento na pilha está relacionado com os demais elementos na mesma camada. Além disso, a pilha é uma estrutura dinâmica, o que significa que os elementos na pilha podem ser adicionados ou removidos à medida que a pilha cresce ou diminui. Portanto, para enfileirar um elemento em uma pilha, você precisará seguir um algoritmo específico para manipular as camadas da pilha e garantir que os elementos estejam organizados corretamente.


### 4.2 - Comparando a coerência

In [ ]:
def gerar_promptCO(texto1, texto2):
    ### texto:
    return f"""TEXTO1:{texto1}\n
TEXTO2:{texto2}\n
Compare o TEXTO1 com o TEXTO2 e identifique quem é mais coerente, utilize o o formato abaixo para apresentar a informação e justificar\n
Mais coerente: <TEXTO>\n
Justificativa: <JUSTIFICATIVA>\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarCO(texto1, texto2):
    # Recupera o prompt
    prompt = gerar_promptCO(texto1, texto2)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto1 = "Como empilhar um elemento em uma fila?"
texto2 = "Como empilhar um elemento em uma pilha?"

avaliarCO(texto1, texto2)

Resposta: 
Mais coerente: TEXTO2

Justificativa: O TEXTO2 utiliza a palavra "pilha" em vez de "fila", o que é o termo mais comum utilizado em programação para referir-se a uma estrutura de dados em que os elementos são empilhados uns sobre os outros. Além disso, o TEXTO2 fornece uma explicação mais clara e detalhada sobre como empilhar um elemento em uma pilha, incluindo informações sobre a estrutura da pilha e como os elementos são empilhados. Em contraste, o TEXTO1 é mais genérico e não fornece detalhes suficientes sobre como empilhar um elemento em uma fila. Por isso, o TEXTO2 é considerado mais coerente.


In [ ]:
texto1 = "Como empilhar um elemento em uma fila?"
texto2 = "Como empilhar um elemento em uma pilha?"

avaliarCO(texto2, texto1)

Resposta: 
Mais coerente: TEXTO2

Justificativa: O TEXTO2 utiliza a palavra "fila" em vez de "pilha", que é a palavra correta para se referir a uma estrutura de dados em que os elementos são adicionados ao final da fila. Além disso, o TEXTO2 fornece uma explicação mais clara e detalhada sobre como empilhar um elemento em uma fila, incluindo informações sobre a estrutura da fila e como os elementos são adicionados e removidos. Em contraste, o TEXTO1 é mais genérico e não fornece detalhes suficientes sobre como empilhar um elemento em uma pilha. Por isso, o TEXTO2 é considerado mais coerente.


### 4.3 - Extração de Informação

In [ ]:
def gerar_promptEI(texto):
    ### texto:
    return f"""TEXTO: {texto}
Dado o texto acima, extraia informações importantes no formato abaixo:
<CHAVE> : <VALOR> \n
Preserve a exata formatação apresentada.
### Resposta: """

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarEI(texto):
    # Recupera o prompt
    prompt = gerar_promptEI(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

avaliarEI(texto)

Resposta: 
* Nome: Alan Mathison Turing
* Data de nascimento: 23 de junho de 1912
* Localização de nascimento: Londres
* Data de falecimento: 7 de junho de 1954
* Localização de falecimento: Wilmslow, Cheshire
* Realizações: matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico
* Reconhecimento: nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais.


## 4.4 - Análise de sentimentos

### 4.4.1 - Análise de sentimentos 1

In [ ]:
def gerar_promptAS1(texto):

  return f"""Classifique os exemplos de declarações a seguir de acordo com as seguintes polaridades Positivo, Negativo e Neutro.
EXEMPLOS:\n {texto}
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarAS1(texto):
    # Recupera o prompt
    prompt = gerar_promptAS1(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "1 - Minha Experiência na loja foi incrível."\
        "2 - Eu acho que podiam melhorar o produto."\
        "3 - O atendimento foi horrível!"\
        "4 - Não volto mais."\
        "5 - Recomendo demais a banoffe. É uma delícia!"

avaliarAS1(texto)

Resposta: 
1 - Positivo
2 - Negativo
3 - Negativo
4 - Negativo
5 - Positivo


### 4.4.2 - Análise de sentimentos 2

In [ ]:
def gerar_promptAS2(texto):

  return  f"""TEXTO: {texto}\n
Classifique os exemplos a seguir de acordo com as polaridades Positivo, Negativo e Neutro.
Utilize o seguinte formato ###DECLARAÇÃO: \n###POLARIDADE: .\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarAS2(texto):
    # Recupera o prompt
    prompt = gerar_promptAS2(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "1 - Minha Experiência na loja foi incrível."\
        "2 - Eu acho que podiam melhorar o produto."\
        "3 - O atendimento foi horrível!"\
        "4 - Não volto mais."\
        "5 - Recomendo demais a banoffe. É uma delícia!"

avaliarAS2(texto)

Resposta: 
Exemplo 1:
###DECLARAÇÃO: Minha Experiência na loja foi incrível.
###POLARIDADE: Positivo

Exemplo 2:
###DECLARAÇÃO: Eu acho que podiam melhorar o produto.
###POLARIDADE: Negativo

Exemplo 3:
###DECLARAÇÃO: O atendimento foi horrível!
###POLARIDADE: Negativo

Exemplo 4:
###DECLARAÇÃO: Não volto mais.
###POLARIDADE: Negativo

Exemplo 5:
###DECLARAÇÃO: Recomendo demais a banoffe. É uma delícia!
###POLARIDADE: Positivo


## 4.5 - Pergunta e resposta

In [ ]:
def gerar_promptPR(texto):
    '''
      Alterações no texto e tabulação impedem a geração da resposta.
    '''
    return f"""Dado o texto a seguir: {texto}\n
            Gere quatro questões em língua portuguesa e suas respectivas respostas utilizando o template abaixo.\n
            Preserve a exata formatação do template apresentado: \n
            PERGUNTA: <PERGUNTA>
            RESPOSTA: <RESPOSTA>
            ### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarPR(texto):
    # Recupera o prompt
    prompt = gerar_promptPR(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

# Tempo de processamento: ~2m20s
avaliarPR(texto)

Resposta: 
PERGUNTA: Como Alan Turing é considerado o pai da ciência da computação teórica e da inteligência artificial?
            RESPOSTA: Alan Turing é considerado o pai da ciência da computação teórica e da inteligência artificial por ter fornecido uma formalização dos conceitos de algoritmo e computação com a máquina de Turing, que pode ser considerada um modelo de um computador de uso geral. Suas realizações no campo da ciência da computação teórica e da inteligência artificial são amplamente reconhecidas e respeitadas.

            PERGUNTA: Por que Alan Turing não foi totalmente reconhecido em seu país de origem durante sua vida?
            RESPOSTA: Alan Turing não foi totalmente reconhecido em seu país de origem durante sua vida por ser homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais. Na época, a homossexualidade era considerada uma doença mental e era ilegal em Inglaterra. Além disso, a Lei de Segredos Oficiais proibia a divulga

# 5 - Exemplos de padrão de pessoa (padrão persona) em prompts

## 5.1 Um matemático

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva como se fosse um professor de matemática. Me explique no idioma portuguesa a importância do teorema de pitágoras.'

avaliarTexto(texto)

Resposta: 
Bem-vindos à minha aula de matemática! Hoje, vamos abordar um dos teoremas mais importantes da área: o Teorema de Pitágoras.
O Teorema de Pitágoras é um dos principais resultados da geometria euclidiana, que estabelece uma relação matemática entre as distâncias de um triângulo retângulo. Em resumo, o teorema afirma que, se um triângulo retângulo é dado, podemos calcular a distância de um de seus catetos, apenas conhecendo as distâncias dos outros dois catetos.
A formulação do teorema é a seguinte:
"Se um triângulo retângulo ABC é dado, onde a é a distância do cateto BC, b é a distância do cateto CA e c é a distância do cateto AB, então:
a^2 + b^2 = c^2"
O que significa que a distância de um cateto de um triângulo retângulo é igual à soma dos quadrados das distâncias dos outros dois catetos.
A importância do Teorema de Pitágoras é enorme, pois permite calcular a distância de um cateto de um triângulo retângulo, apenas conhecendo as distâncias dos outros dois catetos. Isso é f

## 5.2 Um advogado

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva como se fosse um advogado brasileiro especialista em direito penal. '\
        'Pontue de forma resumida as possíveis penas para um caso de lesão corporal leve sem contexto de violência doméstica.'

avaliarTexto(texto)

Resposta: 
Com base nos artigos 129 e 130 do Código Penal Brasileiro, as penas para um caso de lesão corporal leve sem contexto de violência doméstica podem variar de acordo com a grau da lesão e a intenção do agente. Aqui estão as possíveis penas:
* Lesão leve: penas de reclusão de 1 (um) a 3 (três) meses, multa de 100 a 300 reais e/ou servido de trabalho comunitário.
* Lesão moderada: penas de reclusão de 3 (três) a 6 (seis) meses, multa de 300 a 600 reais e/ou servido de trabalho comunitário.
* Lesão grave: penas de reclusão de 6 (seis) a 12 (doze) meses, multa de 600 a 1.200 reais e/ou servido de trabalho comunitário.
Além disso, é importante considerar que, se o lesão for causado por uma conduta dolosa, o agente pode ser punido com penas adicionais, como a perda do direito de exercer determinadas atividades, como o direito de circular em veículos, ou o direito de exercer cargos públicos.
É importante lembrar que essas são apenas possíveis penas, e que o juiz terá o poder de impor 

## 5.3 Um astrofísico

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva em português como se fosse um astrofísico. Me explique por que o universo está expandindo.'

avaliarTexto(texto)

Resposta: 
O universo está expandindo! Isso é um fato comprovado por observações e medições de diversas formas de radiação cósmica, desde a radiação cósmica de fundo (CMB) até a radiação de rádio e óptica de galáxias.
A expansão do universo pode ser entendida como um processo de dilatação das distâncias entre as estrelas e galáxias, o que ocorre devido à aceleração da velocidade das partículas subatômicas, como elétrons e protons. Essa aceleração é causada pela energia escura, um tipo de energia que não pode ser observada diretamente, mas que pode ser detectada pela sua influência sobre a distribuição das massas no universo.
A teoria atual do universo é a teoria do modelo cosmológico estável, que propõe que o universo começou em um estado extremamente quente e denso, e desde então está expandindo-se em um processo que se desenrolla em diferentes etapas. Essa expansão é geralmente considerada como um processo irreversível, ou seja, não há evidências de que o universo esteja se contraind

# 6 - Exemplos de textos estruturados

In [ ]:
def gerar_prompt(texto, entrada=None):
    if entrada:
        return f"""Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{texto}

### Entrada:
{entrada}

### Resposta:"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{texto}

### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto, entrada=None):
    # Recupera o prompt
    prompt = gerar_prompt(texto, entrada)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

## 5.1 - Tarefa simples

In [ ]:
texto = 'Me fale sobre algoritmos.'
avaliarTexto(texto)

Resposta: 
Algoritmos são técnicas de resolução de problemas que utilizam uma sequência de estágios para processar informações. Eles são usados em uma variedade de áreas, como ciência de dados, inteligência artificial, engenharia de software e ciência computacional. Alguns dos algoritmos mais comuns incluem a busca binária, o algoritmo de Fibonacci e o algoritmo de búsqueda de heap. Esses algoritmos são fundamentais para a resolução de problemas complexos e são amplamente utilizados em diversas aplicações.
Abaixo estão alguns exemplos de algoritmos e suas aplicações:
* Algoritmo de Fibonacci: é usado para calcular a série de Fibonacci, que é uma sequência de números em que cada número é a soma dos dois números anteriores. Essa técnica é usada em diversas aplicações, como na modelagem de crescimento populacional e na simulação de sistemas biológicos.
* Algoritmo de b


## 5.2 - Tarefa com entrada

In [ ]:
texto = 'Dada a fórmula química, calcule a massa molar.'

entrada = 'CaCl2'

avaliarTexto(texto, entrada)

Resposta: 
A massa molar de CaCl2 é de 105,5 g/mol.


In [ ]:
texto = 'Faça quatro perguntas sobre a seguinte passagem:'

entrada = 'A anatomia de uma abelha é bastante intrincada. Tem três partes do corpo: a cabeça, o tórax e o abdômen. A cabeça consiste em órgãos sensoriais, três olhos simples e dois olhos compostos e vários apêndices. O tórax tem três pares de pernas e dois pares de asas, enquanto o abdômen contém a maioria dos órgãos da abelha, incluindo o sistema reprodutivo e o sistema digestivo.'

avaliarTexto(texto, entrada)

Resposta: 
Entre as seguintes perguntas, escolha as quatro que você considera mais relevantes ou interessantes para a passagem:
1. Qual é o nome do órgão sensorial na cabeça da abelha?
2. Como muitos pares de pernas tem o tórax da abelha?
3. Qual é o nome do sistema reprodutivo na abdômen da abelha?
4. Qual é o nome dos apêndices na cabeça da abelha?


In [ ]:
texto = 'Analise o documento jurídico fornecido e explique os pontos-chave.'

entrada = 'O seguinte é um trecho de um contrato entre duas partes, rotulado como "Empresa A" e "Empresa B": "A Empresa A concorda em fornecer assistência razoável à Empresa B para garantir a precisão das demonstrações financeiras que fornece. Isso inclui permitir à Empresa um acesso razoável ao pessoal e outros documentos que possam ser necessários para a revisão da Empresa B. A Empresa B concorda em manter o documento fornecido pela Empresa A em confiança e não divulgará as informações a terceiros sem a permissão explícita da Empresa A".'

avaliarTexto(texto, entrada)

Resposta: 
O contrato entre a Empresa A e a Empresa B estabelece uma relação de confiança e segurança em relação às informações financeiras da Empresa B. A Empresa A concorda em fornecer assistência para garantir a precisão das demonstrações financeiras da Empresa B, e a Empresa B concorda em manter as informações fornecidas pela Empresa A em confiança e não divulgará as informações a terceiros sem a permissão explícita da Empresa A.
Entre os pontos-chave do contrato, destacam-se:
* A Empresa A fornece assistência para garantir a precisão das demonstrações financeiras da Empresa B.
* A Empresa B mantém as informações fornecidas pela Empresa A em confiança e não divulga as informações a terceiros sem permissão explícita.
* A Empresa A e a Empresa B estabelecem uma relação de confiança e segurança em relação às informações financeiras da Empresa B.
